In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer, Input, add, dot, maximum, average, multiply, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow import linalg 
import numpy as np
import pandas as pd
from scipy.linalg import circulant
from keras.utils import np_utils
from keras.utils import to_categorical

In [2]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.mean()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [4]:
X = np.array(X)
X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
des=X[:,:13]
body=X[:,13:]

_, des_col = des.shape
_, body_col = body.shape

In [5]:
print(des.shape)
print(body.shape)
print(y.shape)
print(y_1.shape)

(828, 13)
(828, 34)
(828,)
(828, 1)


In [6]:
hid_layer = [100, 100, 50]
print(hid_layer[-1])

50


In [6]:
from keras import backend as K
from keras.layers import Layer

def recurent_matrix(des_shape, body_shape): # circulant matrix 만들어주는 함수

    shape = des_shape.get_shape().as_list() # shape 저장.
    des_set = [] 
    des0 = K.expand_dims(des_shape, 2) # expand_dims: 차원을 늘려주는 함수
    des_set.append(des0)

    body_set = []
    body0 = K.expand_dims(body_shape, 2)
    body_set.append(body0)
    
    
    for i in range(1,shape[1]): # 행렬 생성 shape[1]

        pre=des0[:,(shape[1]-i):,:]
        host=des0[:,0:(shape[1]-i),:]
        des1=tf.concat([pre,host],1)
        des_set.append(des0)

        pre=body0[:,(shape[1]-i):,:]
        host=body0[:,0:(shape[1]-i),:]
        body1=tf.concat([pre,host],1)
        body_set.append(body1)

    vector_des=K.concatenate(des_set) # 3차원 concatenate
    vector_body=K.concatenate(body_set)
    
    multiple_vector1 = K.batch_dot(vector_body, des0)
    multiple_vector2 = K.batch_dot(vector_des, body0)
    
    sum_vector = multiple_vector1 + multiple_vector2

    return sum_vector

In [7]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

def circulant(shape):
    matrix_shape = shape.get_shape().as_list()
    circulant_set = []
    circulant_ = K.expand_dims(shape, 2)
    circulant_set.append(circulant_)

    for i in range(1, matrix_shape[1]):
        pre=circulant_[:,(matrix_shape[1]-i):,:]
        host=circulant_[:,0:(matrix_shape[1]-i),:]
        circulant_1=tf.concat([pre,host],1)
        circulant_set.append(circulant_1)

    vector_ = K.concatenate(circulant_set)

    return vector_

class Circulant_layer(Layer):
    def __init__(self, output_dim, **kwargs):
        #self.output_dim = output_dim
        self.output_dim = output_dim
        super(Circulant_layer, self).__init__(**kwargs)
   
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[0][1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Circulant_layer, self).build(input_shape) 

    def call(self, x):
        assert isinstance(x, list)
        des, body = x
        
        des = K.dot(des, self.kernel) #, self.kernel
        body = K.dot(body, self.kernel)
        
        multiple_vector1 = K.batch_dot(circulant(body), des)
        multiple_vector2 = K.batch_dot(circulant(des), body)
        
        sum_vector = multiple_vector1 + multiple_vector2
        sum_vector_1 = K.dot(sum_vector, self.kernel)
        
        return sum_vector

In [8]:
class MM_Sum:
    def __init__(self, des_shape, body_shape, hid_layer, output_layer):
        self.des_shape = des_shape
        self.body_shape = body_shape
        self.hid_layer = hid_layer  # not used
        self.output_layer = output_layer
        
    def Multi_modal_Sum(self):
        # des modality
        des_input = Input(shape = self.des_shape) # input = 17, des_shape = 17
        des_input_x1 = Dense(self.hid_layer[0], activation = 'relu')(des_input)#(des_input_flat)
        des_input_x1_batch = BatchNormalization()(des_input_x1)
        des_input_x2 = Dense(self.hid_layer[2], activation = 'relu')(des_input_x1_batch)
        des_input_x2_batch = BatchNormalization()(des_input_x2)
        #des_input_x3 = Dense(self.hid_layer[2], activation = 'relu')(des_input_x2_batch)
        #des_input_x3_batch = BatchNormalization()(des_input_x3)

        # body modality
        body_input = Input(shape = self.body_shape) # input = 30,body_shape = 30
        body_input_x1 = Dense(self.hid_layer[0], activation = 'relu')(body_input)#(des_input_flat)
        body_input_x1_batch = BatchNormalization()(body_input_x1)
        body_input_x2 = Dense(self.hid_layer[2], activation = 'relu')(body_input_x1_batch)
        body_input_x2_batch = BatchNormalization()(body_input_x2)
        #body_input_x3 = Dense(self.hid_layer[2], activation = 'relu')(body_input_x2_batch)
        #body_input_x3_batch = BatchNormalization()(body_input_x3)

        
        fusion = Circulant_layer(self.hid_layer[2])([des_input_x2_batch, body_input_x2_batch])
        fusion_out = Dense(self.output_layer, activation = 'sigmoid')(fusion)
        fusion_model = keras.models.Model(inputs = [des_input, body_input], outputs = [fusion_out])
        
        return fusion_model

In [10]:
final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi_modal_Sum()
print(network)

In [11]:
network.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 100)          1400        input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 100)          3500        input_2[0][0]                    
______________________________________________________________________________________________

In [12]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 10, shuffle = True, random_state = 111)
kfold.get_n_splits(X)

10

In [13]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [14]:
dict_y = dict()

class train_model:
    def __init__(self, x, y, model, pd_y, pd_sm_y, pd_cw_y, pd_smcw_y):
        self.X = x
        self.y = y
        self.model = model
        self.pd_y = pd_y
        self.pd_sm_y = pd_sm_y
        self.pd_cw_y = pd_cw_y
        self.pd_smcw_y = pd_smcw_y
        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
                
        return self.pd_y
    
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_sm_y[index] = y_pred[i]
                
        return self.pd_sm_y
    
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_cw_y[index] = y_pred[i]
                
            
        return self.pd_cw_y
    
    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
    
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)
    
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_smcw_y[index] = y_pred[i]    
            
        return self.pd_smcw_y
    

In [118]:
final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi_modal_Sum()
print(network)

f_y, f_sm_y, f_cw_y,f_smcw_y = [], [], [], []
output_fusion = train_model(X, y, network, dict_y, dict_y, dict_y, dict_y)

# sum model training
#f_y = output_fusion.train_based()
#f_sm_y = output_fusion.train_smote()
f_cw_y = output_fusion.train_weight()
#f_smcw_y = output_fusion.train_sm_weight()

TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
Epoch 1/50
21/21 [==============================] - 0s 3ms/step - loss: 0.7604 - accuracy: 0.4819
Epoch 2/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5392 - accuracy: 0.5952
Epoch 3/50
21/21 [==============================] - 0s 3ms/step - loss: 0.4356 - accuracy: 0.6677
Epoch 4/50
21/21 [=======

21/21 [==============================] - 0s 2ms/step - loss: 0.0605 - accuracy: 0.9728
Epoch 46/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0339 - accuracy: 0.9758
Epoch 47/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0339 - accuracy: 0.9879
Epoch 48/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0304 - accuracy: 0.9849
Epoch 49/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0360 - accuracy: 0.9804
Epoch 50/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0315 - accuracy: 0.9924
TEST: [  4  14  17  28  29  35  38  41  44  45  47  48  51  55  61  78  81  88
  90  92  95 108 109 114 116 124 127 128 142 148 151 155 159 161 163 168
 171 174 179 181 191 203 206 207 213 216 217 220 229 235 245 247 253 254
 259 265 267 274 276 286 287 288 292 297 299 305 313 315 333 343 348 352
 362 365 369 372 376 382 387 390 392 393 394 398 400 404 417 424 427 430
 435 445 448 453 454 457 478 479 480 482 494 50

Epoch 14/50
21/21 [==============================] - 0s 3ms/step - loss: 0.0045 - accuracy: 0.9985
Epoch 15/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 0.9970
Epoch 16/50
21/21 [==============================] - 0s 3ms/step - loss: 0.0180 - accuracy: 0.9940
Epoch 17/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 0.9970
Epoch 18/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 0.9985
Epoch 19/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0084 - accuracy: 0.9955
Epoch 20/50
21/21 [==============================] - 0s 3ms/step - loss: 0.0406 - accuracy: 0.9925
Epoch 21/50
21/21 [==============================] - 0s 3ms/step - loss: 0.0351 - accuracy: 0.9894
Epoch 22/50
21/21 [==============================] - 0s 2ms/step - loss: 0.0256 - accuracy: 0.9925
Epoch 23/50
21/21 [==============================] - 0s 3ms/step - loss: 0.0554 - accuracy: 0.9774
Epoch 24/5

In [15]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        print(confusion.ravel())
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy) #pb
        print("Precision(pb) :",mean_precision )
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb

        print("MCC :", mcc)
        print("G-mean :", g_mean)


        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 

        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [119]:
#f_y_result = Evaluation(predict_change(f_y), y)
#f_sm_y_result = Evaluation(f_sm_y, y)
f_cw_y_result = Evaluation(predict_change(f_cw_y), y)
#f_smcw_y_result = Evaluation(f_smcw_y, y)


print("#MCF model 결과 \n")
print("\n-----based model-----\n")
#print(f_y_result.matrix())

print("\n-----smote model-----\n")
#print(f_sm_y_result.matrix())

print("\n-----class weight model-----\n")
print(f_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(f_smcw_y_result.matrix())

#MCF model 결과 


-----based model-----


-----smote model-----


-----class weight model-----

[ 60  11  75 682]
공통 
AUC : 0.9416339516624184
AUPRC : 0.9937181516306834
Optimized precision : 0.48126193096485437

function 사용
Accuracy : 0.8961352657004831
Precision(pb) : 0.9891528008148549
Recall(pb) : 0.6105208797212794
F1 score(pb) : 0.7550270351761427
MCC : 0.565430983021922
G-mean : 0.8725507545385113

Confusion_matrix 사용 
 [[ 60  11]
 [ 75 682]]
Accuracy : 0.8961352657004831
Precision : 0.9841269841269841
Recall : 0.9009247027741083
F1 score : 0.9406896551724139
MCC : 0.5779679114146514
G-mean : 0.8725507545385113
(0.9416339516624184, 0.9937181516306834, 0.48126193096485437, 0.8961352657004831, 0.9891528008148549, 0.6105208797212794, 0.7550270351761427, 0.565430983021922, 0.8725507545385113, 0.8961352657004831, 0.9841269841269841, 0.9009247027741083, 0.9406896551724139, 0.5779679114146514, 0.8725507545385113)

-----smote + class weight model-----



In [21]:
fusion_y_set = []

for i in range(20):
    final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
    network = final_model.Multi_modal_Sum()
    print(network)

    f_y, f_sm_y, f_cw_y,f_smcw_y = [], [], [], []
    output_fusion = train_model(X, y, network, dict_y, dict_y, dict_y, dict_y)

    # sum model training
    f_y = output_fusion.train_based()
    #f_sm_y = output_fusion.train_smote()a
    #f_cw_y = output_fusion.train_weight()
    #f_smcw_y = output_fusion.train_sm_weight()
    
    f_y_result = Evaluation(predict_change(f_y), y_1)
    #f_sm_y_result = Evaluation(f_sm_y, y)
    #f_cw_y_result = Evaluation(predict_change(f_cw_y), y)
    #f_smcw_y_result = Evaluation(predict_change(f_smcw_y), y)
    
    fusion_y_set = np.append(fusion_y_set, f_y_result.matrix())
    print(len(fusion_y_set))
    
fusion_y_np = np.array(fusion_y_set)

fusion_y_np = fusion_y_np.reshape(-1,15)
fusion_y_np.shape

TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
24/24 [==============================] - 0s 5ms/step - loss: 0.6795 - accuracy: 0.5826
Epoch 2/50
24/24 [==============================] - 0s 6ms/step - loss: 0.4820 - accuracy: 0.7745
Epoch 3/50
24/24 [==============================] - 0s 5ms/step - loss: 0.3855 - accuracy: 0.8416
Epoch 4/50
24/24 [==============================] - 0s 6ms/step - loss: 0.3391 - accuracy: 0.8698
Epoch 5/50
24/24 [==============================] - 0s 6ms/step - loss: 0.2806 - accuracy: 0.8738
Epoch 6/50
24/24 [==============================] - 0s 6ms/step - loss: 0.2407 - accuracy: 0.9074
Epoch 7/50
24/24 [==============================] - 0s 5ms/step - lo

24/24 [==============================] - 0s 5ms/step - loss: 0.0257 - accuracy: 0.9906
Epoch 2/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0390 - accuracy: 0.9906
Epoch 3/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0370 - accuracy: 0.9933
Epoch 4/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0169 - accuracy: 0.9919
Epoch 5/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0198 - accuracy: 0.9946
Epoch 6/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0218 - accuracy: 0.9906
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0313 - accuracy: 0.9946
Epoch 8/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0332 - accuracy: 0.9933
Epoch 9/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0151 - accuracy: 0.9919
Epoch 10/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0171 - accuracy: 0.9946
Epoch 11/50
24/24 [===========

24/24 [==============================] - 0s 5ms/step - loss: 0.0222 - accuracy: 0.9919
Epoch 7/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0050 - accuracy: 0.9987
Epoch 8/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0141 - accuracy: 0.9960
Epoch 9/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0245 - accuracy: 0.9906
Epoch 10/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0145 - accuracy: 0.9933
Epoch 11/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0518 - accuracy: 0.9919
Epoch 12/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0168 - accuracy: 0.9933
Epoch 13/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0099 - accuracy: 0.9960
Epoch 14/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0047 - accuracy: 0.9987
Epoch 15/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0075 - accuracy: 0.9973
Epoch 16/50
24/24 [======

24/24 [==============================] - 0s 5ms/step - loss: 0.0055 - accuracy: 0.9987
Epoch 11/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0055 - accuracy: 0.9973
Epoch 12/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0063 - accuracy: 0.9960
Epoch 13/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0038 - accuracy: 0.9987
Epoch 14/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0038 - accuracy: 0.9987
Epoch 15/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 0.9960
Epoch 16/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0199 - accuracy: 0.9960
Epoch 17/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0189 - accuracy: 0.9960
Epoch 18/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 19/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0059 - accuracy: 0.9960
Epoch 20/50
24/24 [===

24/24 [==============================] - 0s 5ms/step - loss: 0.0204 - accuracy: 0.9919
Epoch 10/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0177 - accuracy: 0.9946
Epoch 11/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0245 - accuracy: 0.9933
Epoch 12/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0157 - accuracy: 0.9946
Epoch 13/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0279 - accuracy: 0.9893
Epoch 14/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0223 - accuracy: 0.9919
Epoch 15/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0229 - accuracy: 0.9933
Epoch 16/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0216 - accuracy: 0.9906
Epoch 17/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0230 - accuracy: 0.9919
Epoch 18/50
24/24 [==============================] - 0s 5ms/step - loss: 0.0178 - accuracy: 0.9906
Epoch 19/50
24/24 [===

...

24/24 [==============================] - 0s 6ms/step - loss: 0.0410 - accuracy: 0.9879
Epoch 18/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0302 - accuracy: 0.9893
Epoch 19/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0178 - accuracy: 0.9919
Epoch 20/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0263 - accuracy: 0.9919
Epoch 21/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0100 - accuracy: 0.9946
Epoch 22/50
24/24 [==============================] - 0s 7ms/step - loss: 0.0154 - accuracy: 0.9960
Epoch 23/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0099 - accuracy: 0.9960
Epoch 24/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0174 - accuracy: 0.9933
Epoch 25/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0205 - accuracy: 0.9919
Epoch 26/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0074 - accuracy: 0.9987
Epoch 27/50
24/24 [===

24/24 [==============================] - 0s 6ms/step - loss: 0.0233 - accuracy: 0.9919
Epoch 47/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0175 - accuracy: 0.9933
Epoch 48/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0163 - accuracy: 0.9946
Epoch 49/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0148 - accuracy: 0.9960
Epoch 50/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0104 - accuracy: 0.9960
TEST: [  6  15  18  19  27  30  57  63  76  99 106 107 115 130 143 147 169 182
 184 185 186 188 199 204 208 218 221 224 225 227 238 256 269 278 285 306
 319 320 321 336 337 339 341 347 351 353 358 402 407 408 414 425 438 439
 442 450 459 484 489 495 517 559 564 565 576 584 594 621 655 668 671 674
 676 688 691 696 720 734 746 750 763 798 807]
Epoch 1/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0593 - accuracy: 0.9826
Epoch 2/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0

24/24 [==============================] - 0s 6ms/step - loss: 0.0266 - accuracy: 0.9919
Epoch 23/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0107 - accuracy: 0.9946
Epoch 24/50
24/24 [==============================] - 0s 7ms/step - loss: 0.0092 - accuracy: 0.9946
Epoch 25/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0278 - accuracy: 0.9960
Epoch 26/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0167 - accuracy: 0.9946
Epoch 27/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0139 - accuracy: 0.9933
Epoch 28/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0132 - accuracy: 0.9973
Epoch 29/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0039 - accuracy: 0.9987
Epoch 30/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0063 - accuracy: 0.9960
Epoch 31/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0087 - accuracy: 0.9973
Epoch 32/50
24/24 [===

24/24 [==============================] - 0s 6ms/step - loss: 0.0166 - accuracy: 0.9906
Epoch 2/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0182 - accuracy: 0.9946
Epoch 3/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0152 - accuracy: 0.9960
Epoch 4/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0121 - accuracy: 0.9933
Epoch 5/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0086 - accuracy: 0.9987
Epoch 6/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0063 - accuracy: 0.9987
Epoch 7/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0094 - accuracy: 0.9946
Epoch 8/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0036 - accuracy: 0.9973
Epoch 9/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 10/50
24/24 [==============================] - 0s 6ms/step - loss: 0.0046 - accuracy: 0.9973
Epoch 11/50
24/24 [===========

(20, 15)

In [17]:
def result(data):
    auc = data[:,0]
    auprc = data[:,1]
    op = data[:,2]
    acc = data[:,3]
    precision = data[:,4]
    recall = data[:,5]
    f1 = data[:,6]
    mcc = data[:,7]
    g_mean = data[:,8]
    confu_acc = data[:,9]
    confu_precision = data[:,10]
    confu_recall = data[:,11]
    confu_f1 = data[:,12]
    confu_mcc = data[:,13]
    confu_g_mean = data[:,14]

    print('공통 \nAUC :\n',"[평균 :",format(auc.mean(), ".5f"), "] [표준편차 :",format(auc.std(), ".5f"),"]") # pb
    print("AUPRC :\n", "[평균 :",format(auprc.mean(), ".5f"), "] [표준편차 :",format(auprc.std(), ".5f"),"]")
    print("Optimized precision :\n","[평균 :",format(op.mean(), ".5f"), "] [표준편차 :",format(op.std(), ".5f"),"]")

    print("\nfunction 사용\nAccuracy :\n","[평균 :",format(acc.mean(), ".5f"), "] [표준편차 :",format(acc.std(), ".5f"),"]")
    print("Precision(pb) :\n", "[평균 :",format(precision.mean(), ".5f"), "] [표준편차 :",format(precision.std(), ".5f"),"]")
    print("Recall(pb) :\n", "[평균 :",format(recall.mean(), ".5f"), "] [표준편차 :",format(recall.std(), ".5f"),"]")
    print("F1 score(pb) :\n","[평균 :",format(f1.mean(), ".5f"), "] [표준편차 :",format(f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(mcc.mean(), ".5f"), "] [표준편차 :",format(mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(g_mean.mean(), ".5f"), "] [표준편차 :",format(g_mean.std(), ".5f"),"]")
    
    print("\nConfusion matrix 사용 \nAccuracy :\n", "[평균 :",format(confu_acc.mean(), ".5f"), "] [표준편차 :",format(confu_acc.std(), ".5f"),"]")
    print("Precision :\n", "[평균 :",format(confu_precision.mean(), ".5f"), "] [표준편차 :",format(confu_precision.std(), ".5f"),"]")
    print("Recall :\n", "[평균 :",format(confu_recall.mean(), ".5f"), "] [표준편차 :",format(confu_recall.std(), ".5f"),"]")
    print("F1 score :\n", "[평균 :",format(confu_f1.mean(), ".5f"), "] [표준편차 :",format(confu_f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(confu_mcc.mean(), ".5f"), "] [표준편차 :",format(confu_mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(confu_g_mean.mean(), ".5f"), "] [표준편차 :",format(confu_g_mean.std(), ".5f"),"]")
    

In [22]:
result(fusion_y_np)

공통 
AUC :
 [평균 : 0.95498 ] [표준편차 : 0.00671 ]
AUPRC :
 [평균 : 0.99475 ] [표준편차 : 0.00113 ]
Optimized precision :
 [평균 : 0.41331 ] [표준편차 : 0.01524 ]

function 사용
Accuracy :
 [평균 : 0.94638 ] [표준편차 : 0.00419 ]
Precision(pb) :
 [평균 : 0.98657 ] [표준편차 : 0.00172 ]
Recall(pb) :
 [평균 : 0.77563 ] [표준편차 : 0.01438 ]
F1 score(pb) :
 [평균 : 0.86840 ] [표준편차 : 0.00878 ]
MCC :
 [평균 : 0.66807 ] [표준편차 : 0.02681 ]
G-mean :
 [평균 : 0.83384 ] [표준편차 : 0.02147 ]

Confusion matrix 사용 
Accuracy :
 [평균 : 0.94638 ] [표준편차 : 0.00419 ]
Precision :
 [평균 : 0.97350 ] [표준편차 : 0.00344 ]
Recall :
 [평균 : 0.96770 ] [표준편차 : 0.00384 ]
F1 score :
 [평균 : 0.97059 ] [표준편차 : 0.00231 ]
MCC :
 [평균 : 0.67696 ] [표준편차 : 0.02545 ]
G-mean :
 [평균 : 0.83384 ] [표준편차 : 0.02147 ]
